In [1]:
import pandas as pd # type: ignore
import os

# Отримати список файлів у директоріях
folder_path_idle = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/idle'
folder_path_running = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/running'
folder_path_stairs = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/stairs'
folder_path_walking = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/walking'
file_list_idle = [f for f in os.listdir(folder_path_idle) if f.endswith('.csv')]
file_list_running = [f for f in os.listdir(folder_path_running) if f.endswith('.csv')]
file_list_stairs = [f for f in os.listdir(folder_path_stairs) if f.endswith('.csv')]
file_list_walking = [f for f in os.listdir(folder_path_walking) if f.endswith('.csv')]

# Переконайтеся, що шлях до каталогу правильний
folder_path_idle = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/idle'

# Перелічіть всі CSV-файли в каталозі
file_list_idle = [os.path.join(folder_path_idle, file) for file in os.listdir(folder_path_idle) if file.endswith('.csv')]

# Переконайтеся, що шлях до каталогу правильний
folder_path_running = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/running'

# Перелічіть всі CSV-файли в каталозі
file_list_running = [os.path.join(folder_path_running, file) for file in os.listdir(folder_path_running) if file.endswith('.csv')]

# Переконайтеся, що шлях до каталогу правильний
folder_path_stairs = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/stairs'

# Перелічіть всі CSV-файли в каталозі
file_list_stairs = [os.path.join(folder_path_stairs, file) for file in os.listdir(folder_path_stairs) if file.endswith('.csv')]

# Переконайтеся, що шлях до каталогу правильний
folder_path_walking = 'C:/Users/IJHEA/Documents/Python/HWDS-Rizun/homework/data/walking'

# Перелічіть всі CSV-файли в каталозі
file_list_walking = [os.path.join(folder_path_walking, file) for file in os.listdir(folder_path_walking) if file.endswith('.csv')]

dataframes = []
for file in file_list_idle:
    dataframe = pd.read_csv(file)
    dataframes.append(dataframe)
combined_dataframe_idle = pd.concat(dataframes)
# К-ть рядків у датасеті
num_rows_idle = combined_dataframe_idle.shape[0]
print(f'Кількість рядків: {num_rows_idle}')
# Збереження об'єднаного датасету у файл CSV
combined_dataframe_idle.to_csv('combined_data_idle.csv', index=False)
dataframe1 = pd.read_csv('combined_data_idle.csv')

dataframes = []
for file in file_list_running:
    dataframe = pd.read_csv(file)
    dataframes.append(dataframe)
combined_dataframe_running = pd.concat(dataframes)
# К-ть рядків у датасеті
num_rows_run = combined_dataframe_running.shape[0]
print(f'Кількість рядків: {num_rows_run}')
# Збереження об'єднаного датасету у файл CSV
combined_dataframe_running.to_csv('combined_data_running.csv', index=False)
dataframe2 = pd.read_csv('combined_data_running.csv')

dataframes = []
for file in file_list_stairs:
    dataframe = pd.read_csv(file)
    dataframes.append(dataframe)
combined_dataframe_stairs = pd.concat(dataframes)
# К-ть рядків у датасеті
num_rows_stairs = combined_dataframe_stairs.shape[0]
print(f'Кількість рядків: {num_rows_stairs}')
# Збереження об'єднаного датасету у файл CSV
combined_dataframe_stairs.to_csv('combined_data_stairs.csv', index=False)
dataframe3 = pd.read_csv('combined_data_stairs.csv')

dataframes = []
for file in file_list_walking:
    dataframe = pd.read_csv(file)
    dataframes.append(dataframe)
combined_dataframe_walking = pd.concat(dataframes)
# К-ть рядків у датасеті
num_rows_walk = combined_dataframe_walking.shape[0]
print(f'Кількість рядків: {num_rows_walk}')
# Збереження об'єднаного датасету у файл CSV
combined_dataframe_walking.to_csv('combined_data_walking.csv', index=False)
dataframe4 = pd.read_csv('combined_data_walking.csv')

# Об'єднання датасетів вертикально (за рядком)
combined_dataframe = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4])
# Збереження об'єднаного датасету у файл CSV
combined_dataframe.to_csv('combined_dataframe.csv', index=False)


# Визначення індексів початку у спільному датасеті для кожного типу активності
index_idle = num_rows_idle - num_rows_idle
index_run = num_rows_idle
index_stairs = index_run + num_rows_run
index_walk = index_stairs + num_rows_stairs
print(f'Індекс початку для активності idle: {index_idle}')
print(f'Індекс початку для активності running: {index_run}')
print(f'Індекс початку для активності stairs: {index_stairs}')
print(f'Індекс початку для активності walking: {index_walk}')

Кількість рядків: 31170
Кількість рядків: 102240
Кількість рядків: 4950
Кількість рядків: 55500
Індекс початку для активності idle: 0
Індекс початку для активності running: 31170
Індекс початку для активності stairs: 133410
Індекс початку для активності walking: 138360
